In [1]:
import ast
import glob
import os
import random
import re

import numpy as np
import pandas as pd

from __future__ import annotations
from codealltag_data_processor_v2025 import CodealltagDataProcessor
from concurrent.futures import ThreadPoolExecutor, as_completed
from pandas import DataFrame
from pandas.core.series import Series
from tqdm import tqdm
from typing import Any, Dict, Generator, List, Tuple

In [2]:
cdp_2022 = CodealltagDataProcessor(data_version='20220513', config_path=['codealltag_data_processor.yml'])

In [3]:
sample_df = pd.read_csv('data_syntheticity_dataset_3500_ss.csv', index_col=0)

In [4]:
labels = ['CITY', 'DATE', 'EMAIL', 'FAMILY', 'FEMALE', 'MALE', 'ORG', 'PHONE', 'STREET', 'STREETNO', 'UFID', 'URL', 'USER', 'ZIP']

In [5]:
sample_df_p75 = sample_df[(sample_df.L318BPOS > 0.75) & (sample_df.G29BPOS > 0.75)]

In [6]:
sample_df_p75.reset_index(drop=True, inplace=True)

In [7]:
len(sample_df_p75)

1759

In [8]:
sample_df_p75.columns

Index(['FilePath', 'OT', 'OA', 'OTT', 'OTTL', 'MT5O', 'MT5PT', 'MT5PA',
       'MT5PTT', 'MT5PTTL', 'L318BPO', 'L318BPOS', 'G29BPO', 'G29BPOS'],
      dtype='object')

In [9]:
def prepare_pseudonymized_text_with_llama_and_gemma_output(df: DataFrame, 
                                                           idx: int, 
                                                           cdp: CodealltagDataProcessor) -> Dict[str, Tuple]:
    
    row: Series = df.iloc[idx]
    orig_text: str = row.OT
    l318b_output: str = row.L318BPO
    l318b_adf: DataFrame = cdp.get_annotation_df_with_input_text_and_predicted_text(orig_text, l318b_output, labels)
    l318b_pt: str = cdp.get_pseudonymized_text(orig_text, l318b_adf)
    l318b_ptt: str = " ".join(cdp.tokenize_with_somajo(l318b_pt))
    g29b_output: str = row.G29BPO
    g29b_adf: DataFrame = cdp.get_annotation_df_with_input_text_and_predicted_text(orig_text, g29b_output, labels)
    g29b_pt: str = cdp.get_pseudonymized_text(orig_text, g29b_adf)
    g29b_ptt: str = " ".join(cdp.tokenize_with_somajo(g29b_pt))
    output_tuple = (
        row.FilePath,
        l318b_adf.to_dict(),
        l318b_pt,
        l318b_ptt,
        g29b_adf.to_dict(),
        g29b_pt,
        g29b_ptt
    )
    return {row.FilePath: output_tuple}

In [10]:
def collect_pseudonymized_text_with_llama_and_gemma_output(max_workers: int = 10) -> Generator[Dict[str, Tuple]]:
    with tqdm(total=len(sample_df_p75), smoothing=0) as progress_bar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [
                executor.submit(prepare_pseudonymized_text_with_llama_and_gemma_output, sample_df_p75, idx, cdp_2022)
                for idx in range(0, len(sample_df_p75))
            ]
            for future in as_completed(futures):
                progress_bar.update(1)
                yield future.result()

In [11]:
merged_dict: Dict[str, Tuple] = {}
for result in collect_pseudonymized_text_with_llama_and_gemma_output():
    merged_dict.update(result)

100%|███████████████████████████████████████| 1759/1759 [08:24<00:00,  3.49it/s]


In [18]:
sample_df_p75["L318BPA"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][1])
sample_df_p75["L318BPT"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][2])
sample_df_p75["L318BPTT"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][3])
sample_df_p75["G29BPA"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][4])
sample_df_p75["G29BPT"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][5])
sample_df_p75["G29BPTT"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][6])

/tmp/ipykernel_5523/1201539308.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df_p75["L318BPA"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][1])
/tmp/ipykernel_5523/1201539308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df_p75["L318BPT"] = sample_df_p75["FilePath"].map(lambda fp: merged_dict[fp][2])
/tmp/ipykernel_5523/1201539308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [19]:
pseud_text_df = sample_df_p75.sample(n=1500, random_state=cdp_2022.get_random_seed())
pseud_text_df.reset_index(drop=True, inplace=True)

In [20]:
pseud_text_df.head(n=3)

,FilePath,OT,OA,OTT,OTTL,MT5O,MT5PT,MT5PA,MT5PTT,MT5PTTL,L318BPO,L318BPOS,G29BPO,G29BPOS,L318BPA,L318BPT,L318BPTT,G29BPA,G29BPT,G29BPTT
0,CodEAlltag_pXL_MOVIES/1-/18-/184741.txt,"On Wed, 19 Sep 1998 13:10:08 +0200, Hilda Regg...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","On Wed , 19 Sep 1998 13:10:08 +0200 , Hilda Re...","On O\nWed O\n, O\n19 B-DATE\nSep I-DATE\n1998 ...",DATE: 19 Sep 1998 **17 Okt 1999**; FEMALE: Hil...,"On Wed, 17 Okt 1999 13:10:08 +0200, Verna Aman...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","On Wed , 17 Okt 1999 13:10:08 +0200 , Verna Am...","On O\nWed O\n, O\n17 B-DATE\nOkt I-DATE\n1999 ...",DATE: 19 Sep 1998 **23 Feb 2002**; FEMALE: Hil...,1.0,------------------\nDATE: 19 Sep 1998 **07 Oct...,0.800000,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","On Wed, 23 Feb 2002 13:10:08 +0200, Marieke Ri...","On Wed , 23 Feb 2002 13:10:08 +0200 , Marieke ...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","On Wed, 07 Oct 2023 13:10:08 +0200, Brunhilde ...","On Wed , 07 Oct 2023 13:10:08 +0200 , Brunhild..."
1,CodEAlltag_pXL_TEENS/1-/16-/168935.txt,"Am 15 Jan 2000 12:32:00 +0100, schrieb rcmmoc@...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","Am 15 Jan 2000 12:32:00 +0100 , schrieb rcmmoc...",Am O\n15 B-DATE\nJan I-DATE\n2000 I-DATE\n12:3...,DATE: 15 Jan 2000 **23 Nov 1999**; EMAIL: rcmm...,"Am 23 Nov 1999 12:32:00 +0100, schrieb jthzr@j...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","Am 23 Nov 1999 12:32:00 +0100 , schrieb jthzr@...",Am O\n23 B-DATE\nNov I-DATE\n1999 I-DATE\n12:3...,DATE: 15 Jan 2000 **11. 04. 98**; MALE: Michae...,1.0,-----------------\nDATE: 15 Jan 2000 **18 Dec ...,0.857143,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","Am 11. 04. 98 12:32:00 +0100, schrieb rcmmoc@h...","Am 11. 04. 98 12:32:00 +0100 , schrieb rcmmoc@...","{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...","Am 18 Dec 2003 12:32:00 +0100, schrieb rcmmoc@...","Am 18 Dec 2003 12:32:00 +0100 , schrieb rcmmoc..."
2,CodEAlltag_pXL_TEENS/7-/73011.txt,Papa Leon ist Aktiv in Newsgroups wie Klassisc...,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3'}, 'Lab...",Papa Leon ist Aktiv in Newsgroups wie Klassisc...,Papa O\nLeon B-MALE\nist O\nAktiv O\nin O\nNew...,MALE: Leon **Beat**; PHONE: 68397-410038 **379...,Papa Beat ist Aktiv in Newsgroups wie Klassisc...,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3'}, 'Lab...",Papa Beat ist Aktiv in Newsgroups wie Klassisc...,Papa O\nBeat B-MALE\nist O\nAktiv O\nin O\nNew...,MALE: Leon **Gerhard**; CITY: Newsgroups like ...,1.0,OUTPUT\n-------\nMALE: Leon **Gerhard**; CITY:...,1.000000,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3'}, 'Lab...",Papa Gerhard ist Aktiv in Newsgroups wie Klass...,Papa Gerhard ist Aktiv in Newsgroups wie Klass...,"{'Token_ID': {0: 'T1', 1: 'T2', 2: 'T3', 3: 'T...",Papa Gerhard ist Aktiv in Fichtenwald wie Klas...,Papa Gerhard ist Aktiv in Fichtenwald wie Klas...


In [21]:
def prepare_data_syntheticity_dataset(pseud_text_df: DataFrame, model: str):
    tuples: List[Tuple] = list()
    for idx_o, row_o in pseud_text_df.iterrows():
        tuples.append((row_o.FilePath, row_o.OT, row_o.OTT, "ORIG"))
    for idx_p, row_p in pseud_text_df.iterrows():
        tuples.append((row_p.FilePath, row_p[f'{model}PT'], row_p[f'{model}PTT'], "PSEUD"))
    df = pd.DataFrame(tuples, columns=["FilePath", "Text", "TextTokenized", "Type"])
    df.reset_index(drop=True, inplace=True)
    df.to_csv(f"data_syntheticity_dataset_3000_{model}.csv")

In [22]:
prepare_data_syntheticity_dataset(pseud_text_df, model="MT5")

In [23]:
prepare_data_syntheticity_dataset(pseud_text_df, model="L318B")

In [24]:
prepare_data_syntheticity_dataset(pseud_text_df, model="G29B")

In [25]:
mT5_dataset = pd.read_csv("data_syntheticity_dataset_3000_MT5.csv", index_col=0)

In [26]:
l318b_dataset = pd.read_csv("data_syntheticity_dataset_3000_L318B.csv", index_col=0)

In [27]:
g29b_dataset = pd.read_csv("data_syntheticity_dataset_3000_G29B.csv", index_col=0)

In [28]:
print(mT5_dataset.iloc[0].Text)

On Wed, 19 Sep 1998 13:10:08 +0200, Hilda Reggentin

Der sogenannte "Missionar" wartet noch ein wenig, in etwa bis die
nächste FAQ mit den entsprechenden Verweisen bzgl. drfm und Kritiken
gepostet wurde (damit ihn niemand wie hier geschehen ohne weiteres
anpflaumen kann, weil er seinen überlangen Krempel nicht in drfk
postet), um dann wieder was in drfm zu veröffentlichen. Elsa
Äußerungen waren hierfür der endgültige Auslöser. Nennt es meinetwegen
eine Trotzreaktion. EOT.

cu,
  Luis

P.S.: Wahrscheinlich wird es "Lost Highway" oder die
144-Minuten-Fassung von "The Shining". Ich kann mich nicht
entscheiden.


In [29]:
print(mT5_dataset.iloc[1500].Text)

On Wed, 17 Okt 1999 13:10:08 +0200, Verna Amann

Der sogenannte "Missionar" wartet noch ein wenig, in etwa bis die
nächste FAQ mit den entsprechenden Verweisen bzgl. drfm und Kritiken
gepostet wurde (damit ihn niemand wie hier geschehen ohne weiteres
anpflaumen kann, weil er seinen überlangen Krempel nicht in drfk
postet), um dann wieder was in drfm zu veröffentlichen. Lutz
Äußerungen waren hierfür der endgültige Auslöser. Nennt es meinetwegen
eine Trotzreaktion. EOT.

cu,
  Lars

P.S.: Wahrscheinlich wird es "Lost Highway" oder die
144-Minuten-Fassung von "The Shining". Ich kann mich nicht
entscheiden.


In [30]:
print(l318b_dataset.iloc[0].Text)

On Wed, 19 Sep 1998 13:10:08 +0200, Hilda Reggentin

Der sogenannte "Missionar" wartet noch ein wenig, in etwa bis die
nächste FAQ mit den entsprechenden Verweisen bzgl. drfm und Kritiken
gepostet wurde (damit ihn niemand wie hier geschehen ohne weiteres
anpflaumen kann, weil er seinen überlangen Krempel nicht in drfk
postet), um dann wieder was in drfm zu veröffentlichen. Elsa
Äußerungen waren hierfür der endgültige Auslöser. Nennt es meinetwegen
eine Trotzreaktion. EOT.

cu,
  Luis

P.S.: Wahrscheinlich wird es "Lost Highway" oder die
144-Minuten-Fassung von "The Shining". Ich kann mich nicht
entscheiden.


In [31]:
print(l318b_dataset.iloc[1500].Text)

On Wed, 23 Feb 2002 13:10:08 +0200, Marieke Riemann

Der sogenannte "Missionar" wartet noch ein wenig, in etwa bis die
nächste FAQ mit den entsprechenden Verweisen bzgl. drfm und Kritiken
gepostet wurde (damit ihn niemand wie hier geschehen ohne weiteres
anpflaumen kann, weil er seinen überlangen Krempel nicht in drfk
postet), um dann wieder was in drfm zu veröffentlichen. Elsa
Äußerungen waren hierfür der endgültige Auslöser. Nennt es meinetwegen
eine Trotzreaktion. EOT.

cu,
  Gerson

P.S.: Wahrscheinlich wird es "Lost Highway" oder die
144-Minuten-Fassung von "The Shining". Ich kann mich nicht
entscheiden.


In [32]:
print(g29b_dataset.iloc[0].Text)

On Wed, 19 Sep 1998 13:10:08 +0200, Hilda Reggentin

Der sogenannte "Missionar" wartet noch ein wenig, in etwa bis die
nächste FAQ mit den entsprechenden Verweisen bzgl. drfm und Kritiken
gepostet wurde (damit ihn niemand wie hier geschehen ohne weiteres
anpflaumen kann, weil er seinen überlangen Krempel nicht in drfk
postet), um dann wieder was in drfm zu veröffentlichen. Elsa
Äußerungen waren hierfür der endgültige Auslöser. Nennt es meinetwegen
eine Trotzreaktion. EOT.

cu,
  Luis

P.S.: Wahrscheinlich wird es "Lost Highway" oder die
144-Minuten-Fassung von "The Shining". Ich kann mich nicht
entscheiden.


In [33]:
print(g29b_dataset.iloc[1500].Text)

On Wed, 07 Oct 2023 13:10:08 +0200, Brunhilde Schröder

Der sogenannte "Missionar" wartet noch ein wenig, in etwa bis die
nächste FAQ mit den entsprechenden Verweisen bzgl. drfm und Kritiken
gepostet wurde (damit ihn niemand wie hier geschehen ohne weiteres
anpflaumen kann, weil er seinen überlangen Krempel nicht in drfk
postet), um dann wieder was in drfm zu veröffentlichen. Elsa
Äußerungen waren hierfür der endgültige Auslöser. Nennt es meinetwegen
eine Trotzreaktion. EOT.

cu,
  Klaus

P.S.: Wahrscheinlich wird es "Lost Highway" oder die
144-Minuten-Fassung von "The Shining". Ich kann mich nicht
entscheiden.
